In [1]:
import requests
from requests.auth import HTTPBasicAuth
from flask import Flask, request
import threading
import webbrowser
import time
import pandas as pd
from tqdm import tqdm


# Variáveis Globais
app = Flask(__name__)
authorization_code = None

# Configurações OAuth
client_id = '419759cc09659588aa42c22986968016e4ce2adc'
client_secret = 'ab1097471db5ec489f4e285b35098d95bb27c4469eb6abeabb99fecf46ea'
redirect_url = 'http://localhost:5000/callback'
state = "8bde85dd6e729dcd6e0d01dde003469d"
authorization_url = f'https://www.bling.com.br/Api/v3/oauth/authorize?response_type=code&client_id={client_id}&redirect_uri={redirect_url}&state={state}'
token_url = 'https://www.bling.com.br/Api/v3/oauth/token'
url_base = 'https://www.bling.com.br/Api/v3'
max_pages = 2   # None para todas as páginas
page_limit = 500
id_deposito_escritorio = 863558208

@app.route('/callback')
def callback():
    global authorization_code
    authorization_code = request.args.get('code')
    return 'Authorization code recebido. Você pode fechar esta janela.'

def start_flask_app(start_event):
    start_event.set()
    app.run(port=5000)

def get_authorization_code(client_id: str, redirect_uri: str, state: str) -> str:
    """
    Inicia o processo de obtenção de um código de autorização OAuth2 do Bling.

    Esta função inicia um servidor Flask em uma thread separada para lidar com o callback de autorização,
    constrói uma URL de autorização e abre-a no navegador padrão do usuário. Em seguida, aguarda até que
    o código de autorização seja recebido e definido.

    Parâmetros:
    client_id (str): ID do cliente que está fazendo a requisição.
    redirect_uri (str): URI para onde o usuário será redirecionado após a autorização.
    state (str): Parâmetro opcional para manter o estado entre a requisição e o callback.

    Retorna:
    str: O código de autorização obtido após o usuário autorizar a aplicação.

    Exceções:
    Esta função assume que há uma variável global `authorization_code` que será definida pela aplicação Flask.
    Certifique-se de que o servidor Flask está corretamente configurado para definir essa variável.

    Exemplo de uso:
        client_id = "meu_cliente_id"
        redirect_uri = "https://minhaapp.com/callback"
        state = "estado_arbitrario"

        authorization_code = get_authorization_code(client_id, redirect_uri, state)
        print(f"O código de autorização é: {authorization_code}")
    """
    start_event = threading.Event()
    threading.Thread(target=start_flask_app, args=(start_event,)).start()
    start_event.wait()  # Espera até que o servidor Flask esteja pronto

    url = f'https://www.bling.com.br/Api/v3/oauth/authorize?response_type=code&client_id={client_id}&redirect_uri={redirect_uri}&state={state}'
    webbrowser.open(url)

    # Aguarda até que o `authorization_code` seja definido
    while not authorization_code:
        time.sleep(1)
    
    return authorization_code

def get_access_token(client_id: str, client_secret: str, authorization_code: str) -> dict:
    """
    Obtém um token de acesso OAuth2 do Bling usando um código de autorização.

    Esta função envia uma requisição POST para o endpoint de token do Bling,
    trocando o código de autorização por um token de acesso.

    Parâmetros:
    client_id (str): ID do cliente que está fazendo a requisição.
    client_secret (str): Segredo do cliente usado para autenticação.
    authorization_code (str): Código de autorização obtido após o usuário autorizar a aplicação.

    Retorna:
    dict: A resposta JSON contendo o token de acesso e outros dados relacionados à autenticação.

    Exemplo de uso:
        client_id = "meu_cliente_id"
        client_secret = "meu_segredo_do_cliente"
        authorization_code = "meu_codigo_de_autorizacao"

        access_token_response = get_access_token(client_id, client_secret, authorization_code)
        print(f"O token de acesso é: {access_token_response['access_token']}")
    """
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'Accept': '1.0'
    }
    body = {
        'grant_type': 'authorization_code',
        'code': authorization_code,
        'redirect_uri': 'http://localhost:5000/callback'
    }
    response = requests.post(token_url, headers=headers, data=body, auth=HTTPBasicAuth(client_id, client_secret))
    return response.json()

# Obter o authorization_code
authorization_code = get_authorization_code(client_id, redirect_url, state)

# Obter o access_token usando o authorization_code
token_response = get_access_token(client_id, client_secret, authorization_code)
access_token = token_response.get('access_token')

print(access_token)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [22/May/2024 16:33:04] "GET /callback?code=330e6e321e4664876596863078d8700b1d6f9656&state=8bde85dd6e729dcd6e0d01dde003469d HTTP/1.1" 200 -


6a15eae8d59ece8063323ab315ee73450ca12757


In [9]:
# processo normal sem assync
headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json'
}

tabela_final = []
pagina = 1
while True:
    url = f'https://www.bling.com.br/Api/v3/produtos?pagina={pagina}&limite=500&criterio=1&tipo=T'
    requisicao = requests.get(url, headers=headers)
    informacao = requisicao.json()
    tabela = informacao.get('data', [])
    if len(informacao['data']) < 1:
        break
    prod_df = pd.DataFrame(tabela)
    tabela_final.append(prod_df)
    pagina += 1

df_produtos = pd.concat(tabela_final, ignore_index=True)

df_produtos

,id,idProdutoPai,nome,codigo,preco,tipo,situacao,formato,descricaoCurta,imagemURL
0,16263634776,1.626363e+10,Moletom Lacoste Masculino Hoodie Classic Fit C...,300502,679.99,P,A,S,"<h2 style=""box-sizing: border-box; font-family...",
1,16263634765,1.626363e+10,Moletom Lacoste Masculino Hoodie Classic Fit C...,300501,679.99,P,A,S,"<h2 style=""box-sizing: border-box; font-family...",
2,16263634750,1.626363e+10,Moletom Lacoste Masculino Hoodie Classic Fit C...,300500,679.99,P,A,S,"<h2 style=""box-sizing: border-box; font-family...",
3,16263634736,1.626363e+10,Moletom Lacoste Masculino Hoodie Classic Fit C...,300499,679.99,P,A,S,"<h2 style=""box-sizing: border-box; font-family...",
4,16263634729,1.626363e+10,Moletom Lacoste Masculino Hoodie Classic Fit C...,300498,679.99,P,A,S,"<h2 style=""box-sizing: border-box; font-family...",
...,...,...,...,...,...,...,...,...,...,...
109459,1164833512,1.164833e+09,Camiseta Ralph Lauren Custom Fit Chumbo Tamanh...,1038,189.99,P,A,S,"<h2 style=""box-sizing: inherit; margin: 0px 0p...",https://easyreturn.com.br/modulos/imgprodutos/...
109460,1164833499,1.164833e+09,Camiseta Ralph Lauren Custom Fit Chumbo Tamanh...,1037,189.99,P,A,S,"<h2 style=""box-sizing: inherit; margin: 0px 0p...",https://easyreturn.com.br/modulos/imgprodutos/...
109461,1164833485,1.164833e+09,Camiseta Ralph Lauren Custom Fit Chumbo Tamanh...,1036,189.99,P,A,S,"<h2 style=""box-sizing: inherit; margin: 0px 0p...",https://easyreturn.com.br/modulos/imgprodutos/...
109462,1164833412,NaN,Camiseta Ralph Lauren Custom Fit Chumbo,1036-1,189.99,P,A,V,"<h2 style=""box-sizing: inherit; margin: 0px 0p...",https://easyreturn.com.br/modulos/imgprodutos/...


In [4]:
#processo com assync
import asyncio
import aiohttp
import pandas as pd
import nest_asyncio
from tqdm import tqdm

nest_asyncio.apply()

async def fetch_products(session, pagina, max_retries=10):
    url = f'https://www.bling.com.br/Api/v3/produtos?pagina={pagina}&limite=500&criterio=1&tipo=T'
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json'
    }

    for attempt in range(max_retries):
        async with session.get(url, headers=headers) as response:
            if response.status == 200:
                informacao = await response.json()
                return informacao.get('data', [])
            elif response.status == 429:
                wait_time = 2 ** attempt
                print(f'Erro na requisição: {response.status}. Tentando novamente em {wait_time} segundos...')
                await asyncio.sleep(wait_time)
            else:
                print(f'Erro na requisição: {response.status}')
                return []

async def obter_produtos():
    tabela_final = []
    pagina = 1
    total_paginas = 219  # Baseado na quantidade total de páginas que você mencionou
    pbar = tqdm(total=total_paginas, position=0, leave=True)

    async with aiohttp.ClientSession() as session:
        while True:
            products = await fetch_products(session, pagina)
            if not products:
                break
            prod_df = pd.DataFrame(products)
            tabela_final.append(prod_df)
            pagina += 1
            pbar.update(1)
            await asyncio.sleep(1)  # Espera 1 segundo entre as requisições para evitar limite de taxa

    df_produtos = pd.concat(tabela_final, ignore_index=True)
    return df_produtos

def main():
    # Executar o código assíncrono
    df_produtos = asyncio.get_event_loop().run_until_complete(obter_produtos())
    display(df_produtos)

# Chamar a função main para executar
if __name__ == "__main__":
    main()


220it [07:33,  2.06s/it]                         


,id,idProdutoPai,nome,codigo,preco,tipo,situacao,formato,descricaoCurta,imagemURL
0,16263813201,1.626381e+10,Moletom Lacoste Masculino Crewneck Classic Fit...,300619,749.99,P,A,S,"<h2 style=""font-family: 'Helvetica Neue', Helv...",
1,16263813186,1.626381e+10,Moletom Lacoste Masculino Crewneck Classic Fit...,300618,749.99,P,A,S,"<h2 style=""font-family: 'Helvetica Neue', Helv...",
2,16263813176,1.626381e+10,Moletom Lacoste Masculino Crewneck Classic Fit...,300617,749.99,P,A,S,"<h2 style=""font-family: 'Helvetica Neue', Helv...",
3,16263813165,1.626381e+10,Moletom Lacoste Masculino Crewneck Classic Fit...,300616,749.99,P,A,S,"<h2 style=""font-family: 'Helvetica Neue', Helv...",
4,16263813156,NaN,Moletom Lacoste Masculino Crewneck Classic Fit...,300615,749.99,P,A,V,"<h2 style=""font-family: 'Helvetica Neue', Helv...",
...,...,...,...,...,...,...,...,...,...,...
109581,1164833512,1.164833e+09,Camiseta Ralph Lauren Custom Fit Chumbo Tamanh...,1038,189.99,P,A,S,"<h2 style=""box-sizing: inherit; margin: 0px 0p...",https://easyreturn.com.br/modulos/imgprodutos/...
109582,1164833499,1.164833e+09,Camiseta Ralph Lauren Custom Fit Chumbo Tamanh...,1037,189.99,P,A,S,"<h2 style=""box-sizing: inherit; margin: 0px 0p...",https://easyreturn.com.br/modulos/imgprodutos/...
109583,1164833485,1.164833e+09,Camiseta Ralph Lauren Custom Fit Chumbo Tamanh...,1036,189.99,P,A,S,"<h2 style=""box-sizing: inherit; margin: 0px 0p...",https://easyreturn.com.br/modulos/imgprodutos/...
109584,1164833412,NaN,Camiseta Ralph Lauren Custom Fit Chumbo,1036-1,189.99,P,A,V,"<h2 style=""box-sizing: inherit; margin: 0px 0p...",https://easyreturn.com.br/modulos/imgprodutos/...


In [5]:
import pandas as pd

df_produtos = pd.read_csv('produtdos_bling.csv')

C:\Users\anton\AppData\Local\Temp\ipykernel_20276\1830160586.py:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_produtos = pd.read_csv('produtdos_bling.csv')


In [5]:

# Transformando em um DF e removendo coluna de descrição
lista_produtos = pd.DataFrame(df_produtos)
lista_produtos.drop(columns=['descricaoCurta'], inplace=True)
lista_produtos



,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,id,idProdutoPai,nome,codigo,preco,tipo,situacao,formato,imagemURL
0,0,0,0,0,16263634776,1.626363e+10,Moletom Lacoste Masculino Hoodie Classic Fit C...,300502,679.99,P,A,S,NaN
1,1,1,1,1,16263634765,1.626363e+10,Moletom Lacoste Masculino Hoodie Classic Fit C...,300501,679.99,P,A,S,NaN
2,2,2,2,2,16263634750,1.626363e+10,Moletom Lacoste Masculino Hoodie Classic Fit C...,300500,679.99,P,A,S,NaN
3,3,3,3,3,16263634736,1.626363e+10,Moletom Lacoste Masculino Hoodie Classic Fit C...,300499,679.99,P,A,S,NaN
4,4,4,4,4,16263634729,1.626363e+10,Moletom Lacoste Masculino Hoodie Classic Fit C...,300498,679.99,P,A,S,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
109459,109459,109459,109459,109459,1164833512,1.164833e+09,Camiseta Ralph Lauren Custom Fit Chumbo Tamanh...,1038,189.99,P,A,S,https://easyreturn.com.br/modulos/imgprodutos/...
109460,109460,109460,109460,109460,1164833499,1.164833e+09,Camiseta Ralph Lauren Custom Fit Chumbo Tamanh...,1037,189.99,P,A,S,https://easyreturn.com.br/modulos/imgprodutos/...
109461,109461,109461,109461,109461,1164833485,1.164833e+09,Camiseta Ralph Lauren Custom Fit Chumbo Tamanh...,1036,189.99,P,A,S,https://easyreturn.com.br/modulos/imgprodutos/...
109462,109462,109462,109462,109462,1164833412,NaN,Camiseta Ralph Lauren Custom Fit Chumbo,1036-1,189.99,P,A,V,https://easyreturn.com.br/modulos/imgprodutos/...


In [7]:
id_produto = lista_produtos['id'][0]
codigo_produto = str(lista_produtos['codigo'][0])
print(id_produto, codigo_produto)
len(lista_produtos['codigo'])

16263634776 300502


109464

In [8]:
from tqdm import tqdm

# Barra de progresso
#pbar = tqdm(total=len(lista_produtos['codigo']), position = 0, leave=True)
pbar = tqdm(total=10, position = 0, leave=True)
def consultar_estoque(id_produto, codigo_produto):
    # Configuração da API
    base_url = "https://www.bling.com.br/Api/v3/estoques/saldos"
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json'
    }
    url_estoque = f'{base_url}?idsProdutos[]={id_produto}&codigo={codigo_produto}'
    requisicao = requests.get(url_estoque, headers=headers)
    if requisicao.status_code != 200:
        print(f'Erro na requisição: {requisicao.status_code}')
        return None

    informacao = requisicao.json()
    tabela = informacao.get('data', [])
    if not tabela:
        print('Nenhuma informação encontrada para o produto especificado.')
        return None

    produto_info = tabela[0].get('produto', {})
    saldo_fisico = tabela[0].get('saldoFisicoTotal', 'N/A')
    saldo_virtual = tabela[0].get('saldoVirtualTotal', 'N/A')
    depositos_info = tabela[0].get('depositos', [])

    # Criar um DF apenas com a primeira linha dos dados
    deposito_info_0 = [depositos_info[0]]  # Pega somente a primeira linha
    estoque_df = pd.DataFrame(deposito_info_0)

    estoque_df['id_produto'] = id_produto
    estoque_df['codigo'] = codigo_produto
    estoque_df['saldo'] = saldo_virtual

    return estoque_df


tabela_estoque = []
i = 0
while i != 10:
    pbar.update()
    info = consultar_estoque(lista_produtos['id'][i], codigo_produto = lista_produtos['codigo'][i])
    df_estoque = pd.DataFrame(info)
    tabela_estoque.append(df_estoque)
    i += 1

df_estoque = pd.concat(tabela_estoque, ignore_index=True)
display(df_estoque)


100%|██████████| 10/10 [00:11<00:00,  1.14s/it]

,id,saldoFisico,saldoVirtual,id_produto,codigo,saldo
0,863558208,0,0,16263634776,300502,0
1,863558208,0,0,16263634765,300501,0
2,863558208,0,0,16263634750,300500,0
3,863558208,0,0,16263634736,300499,0
4,863558208,0,0,16263634729,300498,0
5,863558208,0,0,16263634713,300497,0
6,863558208,0,0,16263634699,300496,0
7,863558208,0,0,16263634680,300495,0
8,863558208,0,0,16263632457,300494,0
9,863558208,0,0,16263632455,300493,0


In [ ]:
import asyncio
import aiohttp
import pandas as pd
import nest_asyncio
from tqdm import tqdm

pbar = tqdm(total=len(lista_produtos['codigo']), position = 0, leave=True)

nest_asyncio.apply()

async def consultar_estoque(session, id_produto, codigo_produto, max_retries=5):
    base_url = "https://www.bling.com.br/Api/v3/estoques/saldos"
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json'
    }
    url_estoque = f'{base_url}?idsProdutos[]={id_produto}&codigo={codigo_produto}'

    for attempt in range(max_retries):
        async with session.get(url_estoque, headers=headers) as response:
            if response.status == 200:
                informacao = await response.json()
                tabela = informacao.get('data', [])
                if not tabela:
                    print('Nenhuma informação encontrada para o produto especificado.')
                    return None

                produto_info = tabela[0].get('produto', {})
                saldo_fisico = tabela[0].get('saldoFisicoTotal', 'N/A')
                saldo_virtual = tabela[0].get('saldoVirtualTotal', 'N/A')
                depositos_info = tabela[0].get('depositos', [])

                # Criar um DF apenas com a primeira linha dos dados
                deposito_info_0 = [depositos_info[0]]  # Pega somente a primeira linha
                estoque_df = pd.DataFrame(deposito_info_0)

                estoque_df['id_produto'] = id_produto
                estoque_df['codigo'] = codigo_produto
                estoque_df['saldo'] = saldo_virtual

                return estoque_df
            elif response.status == 429:
                wait_time = 2 ** attempt
                print(f'Erro na requisição: {response.status}. Tentando novamente em {wait_time} segundos...')
                await asyncio.sleep(wait_time)
            else:
                print(f'Erro na requisição: {response.status}')
                return None
    return None

async def main():
    tabela_estoque = []
    async with aiohttp.ClientSession() as session:
        tasks = []
        for i in range(109464):
            pbar.update()
            id_produto = lista_produtos['id'][i]
            codigo_produto = lista_produtos['codigo'][i]
            tasks.append(consultar_estoque(session, id_produto, codigo_produto))
            await asyncio.sleep(1)  # Espera 1 segundo entre as requisições para diminuir a taxa de requisições

        resultados = await asyncio.gather(*tasks)

    for df in resultados:
        if df is not None:
            tabela_estoque.append(df)

    df_estoque = pd.concat(tabela_estoque, ignore_index=True)
    display(df_estoque)

# Executar o código assíncrono
asyncio.get_event_loop().run_until_complete(main())
